Libraries

In [1]:
import pandas as pd
import numpy as np
import os

Combining csv files for tourism data

In [2]:
df_list = []
root = 'data/tourism/'
for f in os.listdir(root):
    df = pd.read_csv(root + f, skiprows = 4)
    columns = df.columns.tolist()
    years = [str(i) for i in range(1995, 2021)] # years we are merging
    df_list.append(df[columns[:4] + years])
    
tourism_df = pd.concat(df_list)
tourism_df = tourism_df.rename(columns = {'Country Name': 'Country', 'Indicator Name': 'Indicator'})

Cleaning inflation data

In [3]:
removed_content_count = 0

with open('data/inflation/inflation_data.csv', 'rb') as file:
    content = file.read()
    cleaned_content = bytearray()

    for byte in content:
        try:
            decoded_byte = byte.to_bytes(1, byteorder='big').decode('utf-8')
            cleaned_content.append(byte)
        except UnicodeDecodeError: # removing problem bytes before reading csv
            removed_content_count += 1

with open('data/inflation/inflation_data_cleaned.csv', 'wb') as cleaned_file: # writing cleaned file
    cleaned_file.write(cleaned_content)

print(f"Removed {removed_content_count} instances of problematic content.")

inflation_df = pd.read_csv('data/inflation/inflation_data_cleaned.csv', encoding='utf-8')
inflation_df = inflation_df.rename(columns = {'Series Name': 'Indicator'})

Removed 9 instances of problematic content.


Filtering to common columns and concatenating dfs

In [9]:
inflation_cols = inflation_df.columns.tolist()
tourism_cols = tourism_df.columns.tolist()
common_cols = set(inflation_cols) & set(tourism_cols) # using common columns before concatenating our datasets
sorted_common_cols = sorted(common_cols, key=lambda x: (x.isdigit(), int(x) if x.isdigit() else x.lower()))

tourism_df = tourism_df[sorted_common_cols]
inflation_df = inflation_df[sorted_common_cols]
final_df = pd.concat([tourism_df, inflation_df]).sort_values(by = 'Country').reset_index(drop = True)

Exploring the data

In [16]:
nan_count_per_column = final_df.isna().sum()
nan_count_per_row = final_df.isna().sum(axis=1)
print(nan_count_per_column), print(nan_count_per_row)

Country            0
Country Code       0
Indicator          0
1995            1286
1996            1221
1997            1182
1998            1160
1999            1124
2000            1094
2001            1037
2002             988
2003             934
2004             919
2005             847
2006             836
2007             773
2008             762
2009             756
2010             750
2011             748
2012             727
2013             726
2014             671
2015             656
2016             643
2017             694
2018             712
2019             834
2020            1320
dtype: int64
0       12
1       13
2       11
3       13
4       13
        ..
3172    14
3173    14
3174    14
3175     0
3176    14
Length: 3177, dtype: int64


(None, None)

In [18]:
final_df['Indicator'].unique()

array(['Energy Consumer Price Inflation',
       'International tourism, expenditures for passenger transport items (current US$)',
       'Food Consumer Price Inflation',
       'International tourism, receipts for passenger transport items (current US$)',
       'International tourism, receipts (% of total exports)',
       'International tourism, receipts (current US$)',
       'International tourism, number of departures',
       'International tourism, receipts for travel items (current US$)',
       'International tourism, expenditures (current US$)',
       'International tourism, number of arrivals',
       'Headline Consumer Price Inflation',
       'Official Core Consumer Price Inflation',
       'Producer Price Inflation'], dtype=object)

In [20]:
tourism_df = final_df[final_df['Indicator'] == 'International tourism, number of departures']
tourism_df

,Country,Country Code,Indicator,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
6,Afghanistan,AFG,"International tourism, number of departures",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Africa Eastern and Southern,AFE,"International tourism, number of departures",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Africa Western and Central,AFW,"International tourism, number of departures",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,Albania,ALB,"International tourism, number of departures",NaN,NaN,NaN,NaN,NaN,NaN,9.550000e+05,...,4.120000e+06,3.959000e+06,3.928000e+06,4.146000e+06,4.504000e+06,4.852000e+06,5.186000e+06,5.415000e+06,5.922000e+06,2907000.0
54,Algeria,DZA,"International tourism, number of departures",1090000.0,874000.0,8.270000e+05,8.790000e+05,9.030000e+05,1.006000e+06,1.190000e+06,...,1.715000e+06,1.911000e+06,2.136000e+06,2.839000e+06,3.638000e+06,4.530000e+06,5.058000e+06,5.610000e+06,5.732000e+06,1386000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124,West Bank and Gaza,PSE,"International tourism, number of departures",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3138,World,WLD,"International tourism, number of departures",NaN,NaN,9.300660e+08,9.821260e+08,1.043185e+09,1.122419e+09,1.121399e+09,...,1.504559e+09,1.564353e+09,1.641229e+09,1.651310e+09,1.744980e+09,1.820357e+09,1.934268e+09,1.991161e+09,2.034432e+09,NaN
3143,"Yemen, Rep.",YEM,"International tourism, number of departures",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3163,Zambia,ZMB,"International tourism, number of departures",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
num_countries = len(final_df['Country'].unique())

In [12]:
num_countries

283

In [13]:
final_df

,Country,Country Code,Indicator,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Afghanistan,AFG,Energy Consumer Price Inflation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.236000e+01,1.181000e+01,8.800000e+00,-7.790000e+00,-4.250000e+00,2.070000e+00,4.440000e+00,1.470000e+00,-2.490000e+00,NaN
1,Afghanistan,AFG,"International tourism, expenditures for passen...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.000000e+06,5.000000e+06,6.000000e+06,3.000000e+06,4.000000e+06,1.000000e+07,1.200000e+07,2.300000e+07,3.900000e+07,1.900000e+07
2,Afghanistan,AFG,Food Consumer Price Inflation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.067000e+01,4.730000e+00,7.570000e+00,7.740000e+00,-8.400000e-01,5.670000e+00,6.940000e+00,-1.050000e+00,3.790000e+00,5.830000e+00
3,Afghanistan,AFG,"International tourism, receipts for passenger ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.000000e+07,6.500000e+07,4.100000e+07,1.200000e+07,7.000000e+06,1.300000e+07,1.400000e+07,2.200000e+07,1.300000e+07,1.000000e+07
4,Afghanistan,AFG,"International tourism, receipts (% of total ex...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.396047e+00,7.170887e+00,1.294099e+01,6.094592e+00,6.067873e+00,5.457683e+00,1.387809e+00,3.106609e+00,5.607753e+00,5.080248e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172,Zimbabwe,ZWE,"International tourism, receipts (% of total ex...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.367463e+00,4.042161e+00,4.394619e+00,4.597858e+00,4.818169e+00,4.778703e+00,3.405013e+00,3.688565e+00,5.411115e+00,1.253967e+00
3173,Zimbabwe,ZWE,"International tourism, expenditures for passen...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.900000e+07,1.150000e+08,1.040000e+08,1.090000e+08,8.100000e+07,7.900000e+07,2.200000e+07,2.100000e+07,5.300000e+07,2.200000e+07
3174,Zimbabwe,ZWE,"International tourism, receipts for travel ite...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.520000e+08,1.610000e+08,1.700000e+08,1.720000e+08,1.750000e+08,1.770000e+08,1.490000e+08,1.820000e+08,2.790000e+08,6.300000e+07
3175,Zimbabwe,ZWE,Headline Consumer Price Inflation,15.23,6.01,-0.96,-28.02,-13.43,4.47,-37.2,...,3.470000e+00,3.720000e+00,1.630000e+00,-2.100000e-01,-2.410000e+00,-1.570000e+00,9.100000e-01,1.061000e+01,2.552900e+02,5.572100e+02
